### Importing the Libraries

In [23]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [24]:
tf.__version__ 

'2.13.0-dev20230404'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [25]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [26]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [27]:
training_set = train_datagen.flow_from_directory("C:\\Users\\lenovo\\Desktop\\Sign-Language-To-Text-Conversion-main\\dataSet\\trainingData",                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [28]:
test_set = test_datagen.flow_from_directory("C:\\Users\\lenovo\\Desktop\\Sign-Language-To-Text-Conversion-main\\dataSet\\testingData",
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [29]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [30]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [31]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [32]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [33]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [34]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [35]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [36]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [37]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5
1285/1285 [==============================] - 301s 233ms/step - loss: 2.1732 - accuracy: 0.3099 - val_loss: 0.6633 - val_accuracy: 0.7533
Epoch 2/5
1285/1285 [==============================] - 296s 230ms/step - loss: 1.0735 - accuracy: 0.6256 - val_loss: 0.2367 - val_accuracy: 0.9349
Epoch 3/5
1285/1285 [==============================] - 283s 220ms/step - loss: 0.7756 - accuracy: 0.7325 - val_loss: 0.1222 - val_accuracy: 0.9726
Epoch 4/5
1285/1285 [==============================] - 288s 224ms/step - loss: 0.6237 - accuracy: 0.7861 - val_loss: 0.0627 - val_accuracy: 0.9885
Epoch 5/5
1285/1285 [==============================] - 283s 220ms/step - loss: 0.5335 - accuracy: 0.8209 - val_loss: 0.0408 - val_accuracy: 0.9920


#### Saving the Model

In [38]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
